# Library/Comon

In [1]:
try:
  ! pip install dill
  import dill
except:
  import dill


In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn

import seaborn as sns
import warnings; warnings.filterwarnings(action='once')

from tqdm import tqdm

import datetime

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier

import random
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [4]:
# My data

DIR = '/content/drive/MyDrive/Colab Notebooks/ROBO/My_Feature_from_NET/'

# if torch.cuda.is_available():  
#   device = "cuda:0" 
# else:  
#   device = "cpu"

# from google.colab import drive
# drive.mount('/content/drive')

# # insted
# DIR = ('F:/Мой диск/Colab Notebooks/ROBO/My_Feature_from_NET/')

In [5]:
def profits(all):

  all['profit_GEP'] = (all['Cl_firs_tomor']/all['Cl_last_today'] - 1)*100
  all['profit_GEP_label'] = np.where(all['profit_GEP'] > 0, 1, 0)
  
  all['profit_1hour'] = np.where(all['Symbol'].shift(-1) == all['Symbol'].shift(0), (all['Close'].shift(-1)/all['Close'] - 1)*100, 0)

  
  return all, 

# add dificult profit

def ad_profit_to_13(all, i, ii):
  close_time = pd.pivot_table(all,
                index = [all.DateTime.dt.date, all.Symbol],
                columns = all.DateTime.dt.time,
                values = 'Close'
                ).reset_index()
  close_time.sort_values(by = ['Symbol', 'DateTime'], inplace=True)
  
  close_time.dropna(inplace=True)
  close_time.reset_index(inplace = True, drop = True)
  close_time['DateTime'] = pd.to_datetime(close_time['DateTime'])
  close_time['Date'] = close_time['DateTime'].dt.date
  close_time['Date'] = pd.to_datetime(close_time['Date'])
  close_time.columns = [str(i) for i in close_time.columns]

  pr_name = 'profit_' + str(i) + '_to_' + str(ii)
  close_time[pr_name] = np.where(close_time['Symbol'] == close_time['Symbol'].shift(-1), 100*(close_time[i].shift(-1)/close_time[ii] - 1), 0)
  close_time[pr_name] = np.where(close_time[pr_name] > 20, 20, np.where(close_time[pr_name] < -20, -20, close_time[pr_name]))
  pr_label = pr_name + '_label'
  close_time[pr_label] = np.where(close_time[pr_name] > 0, 1, 0)
  return pd.merge(close_time[['Date', 'Symbol', pr_name, pr_label]], all, on = ['Date', 'Symbol'], how = 'right'), pr_name, pr_label


def lfl(df_f_all):
  data_np = np.array(df_f_all['Date'].dt.date)
  close_np = np.array(df_f_all['Close'])
  # high_np = np.array(df_f_all['High'])
  low_np = np.array(df_f_all['Low'])
  # print(data_np.size == close_np.size == high_np.size == low_np.size)
  k1 = []
  Cl_last_yeast = np.zeros(shape=data_np.size)
  Cl_last_today = np.zeros(shape=data_np.size)
  Cl_firs_today = np.zeros(shape=data_np.size)
  Cl_firs_tomor = np.zeros(shape=data_np.size)
  Min_day = np.zeros(shape=data_np.size)
  Max_day = np.zeros(shape=data_np.size)

  for i in range(data_np.size):
    if i == 0:
      Cl_last_yeast[i] = close_np[i]
    elif data_np[i] == data_np[i-1]:
      Cl_last_yeast[i] = Cl_last_yeast[i-1]
    else: 
      Cl_last_yeast[i] = close_np[i-1]

  df_f_all['Cl_last_yeast'] = Cl_last_yeast

  # #Short
  k1 = []

  for i in range(data_np.size-1, -1, -1):
    # print(i)
    if i == data_np.size - 1:
      Cl_last_today[i]  =  close_np[i]
      k = 1

    elif data_np[i] == data_np[i+1]:

      Cl_last_today[i]  =  Cl_last_today[i+1]
      k += 1

    else:

      Cl_last_today[i]  = close_np[i]
      k = 1
    k1.append(k)

  df_f_all['Short'] = k1[::-1]  
  df_f_all['Cl_last_today'] = Cl_last_today

  # Long
  k1 = []

  for i in range(data_np.size):
    if i == 0:
      Cl_firs_today[i]   =  close_np[i]
      k = 1

    elif data_np[i] == data_np[i-1]:

      Cl_firs_today[i]   =  Cl_firs_today[i-1]
      k += 1

    else:

      Cl_firs_today[i]  = close_np[i]
      k = 1
    k1.append(k)

  df_f_all['Long'] = k1
  df_f_all['Cl_firs_today'] = Cl_firs_today 

  # Clo_tomor

  for i in range(data_np.size-1, -1, -1):

    if i == data_np.size - 1:
      Cl_firs_tomor[i]  =  close_np[i]

    elif data_np[i] == data_np[i+1]:
      Cl_firs_tomor[i]  =  Cl_firs_tomor[i+1]

    else:
      Cl_firs_tomor[i]  = close_np[i+1]
  
  df_f_all['Cl_firs_tomor'] = Cl_firs_tomor


  # MinD

  for i in range(data_np.size):
    if i == 0:
      Min_day[i] = low_np[i]

    elif data_np[i] == data_np[i-1]:
      if low_np[i] < Min_day[i-1]:
        Min_day[i] = low_np[i]
      else:
        Min_day[i] = Min_day[i-1]

    else: 
      Min_day[i] = low_np[i]

  df_f_all['Min_day'] = Min_day



  # MaxD

  # for i in range(data_np.size):
  #   if i == 0:
  #     Max_day[i] = high_np[i]

  #   elif data_np[i] == data_np[i-1]:
  #     if high_np[i] > Max_day[i-1]:
  #       Max_day[i] = high_np[i]
  #     else:
  #       Max_day[i] = Max_day[i-1]

  #   else: 
  #     Max_day[i] = high_np[i]

  # df_f_all['Max_day'] = Max_day

def use_lfl(df_f_all):
  df_all = pd.DataFrame()
  for i in df_f_all['Symbol'].unique():
    df = df_f_all[df_f_all['Symbol'] == i].copy()
    lfl(df)
    df_all = pd.concat([df_all, df])
    df_all.reset_index(inplace=True, drop = True)
  return df_all


def hide(df):
  df['Cl_min_day']      = (df['Close']/df['Min_day']-1)*100

  df['Proft1000']       = (df['Cl_firs_tomor']/df['Close'].shift(0)-1)*100
  
  return df

# def data(all):
#   all['Date'] = pd.to_datetime(all['Date'])
#   all['DateTime'] = all['Date']
#   all['DateTime'] = pd.to_datetime(all['DateTime'])
#   all['Date'] = all['DateTime'].dt.date
#   all['Date'] = pd.to_datetime(all['Date'])
#   # all = all[(all['DateTime'].dt.time >= datetime.time(10,10))&(all['DateTime'].dt.time <= datetime.time(18,30))]
#   # all = all[(all['DateTime'].dt.year >=2011)]
#   return all  

def generate(data_, range_, cols = ['Cl_min_day']):
  features = []
  for col in cols:
    empty = np.empty(shape = (data_.shape[0], range_))
    for sym in tqdm(data_['Symbol'].unique()):
      np_col = 0
      df_sym = data_[data_['Symbol'] == sym]  
      for i in (range(range_)):
        empty[df_sym.index[0]:df_sym.index[-1] + 1, np_col] = df_sym[col].shift(i)
        np_col +=1
    feature = [str(col) + '_' + str(i) for i in range(range_)]
    features.append(feature)
    data_[feature] = empty
  
  return data_


# Load the dataset

In [6]:
file = 'all_hour1800_300422.csv'
# all_ = pd.read_csv(DIR + 'Data/all/' + file, index_col=[0])
all_ = pd.read_csv(file, index_col=[0])

# all[['Date',	'Symbol', 'Open',	'High',	'Low',	'Close',	'Volume']].to_csv(DIR + 'Data/all/' + 'all_hour1800_300422.csv')


In [7]:
all = all_.iloc[-100000:, :].reset_index(drop = True)
all.drop(columns = ['Open', 'High', 'Volume'], inplace = True)

y_pers = np.where(all['Symbol'].shift(-1) == all['Symbol'].shift(0), (all['Close'].shift(-1)/all['Close'] - 1)*100, 0)
y = np.where(y_pers  > 0, 1, 0)

In [8]:
# X_train, X_test, y_train, y_test = train_test_split(all, y)
X_train = all[:int(len(all)*0.75)]
X_test = all[int(len(all)*0.75):]
X_test = X_test.reset_index(drop = True)

y_train = y[:int(len(all)*0.75)]
y_test = y[int(len(all)*0.75):]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(75000, 4) (75000,) (25000, 4) (25000,)


## Pipline

In [9]:
# [str(col) + '_' + str(i) for i in range(range_)]

In [10]:
# use_lfl(data(X_train))

In [11]:
def generate(data_, range_, cols = ['Cl_min_day']):
  features = []
  for col in cols:
    empty = np.empty(shape = (data_.shape[0], range_))
    for sym in tqdm(data_['Symbol'].unique()):
      np_col = 0
      df_sym = data_[data_['Symbol'] == sym]  
      for i in (range(range_)):
        empty[df_sym.index[0]:df_sym.index[-1] + 1, np_col] = df_sym[col].shift(i)
        np_col +=1
    feature = [str(col) + '_' + str(i) for i in range(range_)]
    # features.append(feature)
    # data_[feature] = empty
    # data_ = data_.join(pd.DataFrame(empty, columns=feature))
  
  return data_.join(pd.DataFrame(empty, columns=feature))

In [52]:
class ExperimentalTransformer(BaseEstimator, TransformerMixin):

  def fit(self, X, y = None):

    return self

  def transform(self, X, y = None):

    X_ = X.copy() 
    print(X_)
    X_ = self.data(X_)
    X_ = self.use_lfl(X_)
    X_ = self.hide(X_)
    X_ = self.generate(X_, 9, ['Cl_min_day'])[['Cl_min_day_' + str(i) for i in range(9)]].fillna(0)

    return X_
  def generate(self, data_, range_, cols = ['Cl_min_day']):
      features = []
      for col in cols:
        empty = np.empty(shape = (data_.shape[0], range_))
        for sym in tqdm(data_['Symbol'].unique()):
          np_col = 0
          df_sym = data_[data_['Symbol'] == sym]  
          for i in (range(range_)):
            empty[df_sym.index[0]:df_sym.index[-1] + 1, np_col] = df_sym[col].shift(i)
            np_col +=1
        feature = [str(col) + '_' + str(i) for i in range(range_)]
        # features.append(feature)
        # data_[feature] = empty
        # data_ = data_.join(pd.DataFrame(empty, columns=feature))

      return data_.join(pd.DataFrame(empty, columns=feature))
  def hide(self, df):
      df['Cl_min_day']      = (df['Close']/df['Min_day']-1)*100

      df['Proft1000']       = (df['Cl_firs_tomor']/df['Close'].shift(0)-1)*100

      return df

  def data(self, all):
      all['Date'] = pd.to_datetime(all['Date'])
      all['DateTime'] = all['Date']
      all['DateTime'] = pd.to_datetime(all['DateTime'])
      all['Date'] = all['DateTime'].dt.date
      all['Date'] = pd.to_datetime(all['Date'])
      # all = all[(all['DateTime'].dt.time >= datetime.time(10,10))&(all['DateTime'].dt.time <= datetime.time(18,30))]
      # all = all[(all['DateTime'].dt.year >=2011)]
      return all  
  def use_lfl(self, df_f_all):
      df_all = pd.DataFrame()
      for i in df_f_all['Symbol'].unique():
        df = df_f_all[df_f_all['Symbol'] == i].copy()
        self.lfl(df)
        df_all = pd.concat([df_all, df])
        df_all.reset_index(inplace=True, drop = True)
      return df_all

  def lfl(self, df_f_all):
      data_np = np.array(df_f_all['Date'].dt.date)
      close_np = np.array(df_f_all['Close'])
      # high_np = np.array(df_f_all['High'])
      low_np = np.array(df_f_all['Low'])
      # print(data_np.size == close_np.size == high_np.size == low_np.size)
      k1 = []
      Cl_last_yeast = np.zeros(shape=data_np.size)
      Cl_last_today = np.zeros(shape=data_np.size)
      Cl_firs_today = np.zeros(shape=data_np.size)
      Cl_firs_tomor = np.zeros(shape=data_np.size)
      Min_day = np.zeros(shape=data_np.size)
      Max_day = np.zeros(shape=data_np.size)

      for i in range(data_np.size):
        if i == 0:
          Cl_last_yeast[i] = close_np[i]
        elif data_np[i] == data_np[i-1]:
          Cl_last_yeast[i] = Cl_last_yeast[i-1]
        else: 
          Cl_last_yeast[i] = close_np[i-1]

      df_f_all['Cl_last_yeast'] = Cl_last_yeast

      # #Short
      k1 = []

      for i in range(data_np.size-1, -1, -1):
        # print(i)
        if i == data_np.size - 1:
          Cl_last_today[i]  =  close_np[i]
          k = 1

        elif data_np[i] == data_np[i+1]:

          Cl_last_today[i]  =  Cl_last_today[i+1]
          k += 1

        else:

          Cl_last_today[i]  = close_np[i]
          k = 1
        k1.append(k)

      df_f_all['Short'] = k1[::-1]  
      df_f_all['Cl_last_today'] = Cl_last_today

      # Long
      k1 = []

      for i in range(data_np.size):
        if i == 0:
          Cl_firs_today[i]   =  close_np[i]
          k = 1

        elif data_np[i] == data_np[i-1]:

          Cl_firs_today[i]   =  Cl_firs_today[i-1]
          k += 1

        else:

          Cl_firs_today[i]  = close_np[i]
          k = 1
        k1.append(k)

      df_f_all['Long'] = k1
      df_f_all['Cl_firs_today'] = Cl_firs_today 

      # Clo_tomor

      for i in range(data_np.size-1, -1, -1):

        if i == data_np.size - 1:
          Cl_firs_tomor[i]  =  close_np[i]

        elif data_np[i] == data_np[i+1]:
          Cl_firs_tomor[i]  =  Cl_firs_tomor[i+1]

        else:
          Cl_firs_tomor[i]  = close_np[i+1]

      df_f_all['Cl_firs_tomor'] = Cl_firs_tomor


      # MinD

      for i in range(data_np.size):
        if i == 0:
          Min_day[i] = low_np[i]

        elif data_np[i] == data_np[i-1]:
          if low_np[i] < Min_day[i-1]:
            Min_day[i] = low_np[i]
          else:
            Min_day[i] = Min_day[i-1]

        else: 
          Min_day[i] = low_np[i]

        df_f_all['Min_day'] = Min_day

In [53]:
# with input transformation
print("create pipeline 2")
pipe2 = Pipeline(steps=[
                       ('transform_data', ExperimentalTransformer()),    # this will trigger a call to __init__
                       ('linear_model', LinearRegression())
                      #  ('GB', GradientBoostingClassifier())
])

print("fit pipeline 2")
pipe2.fit(X_train, y_train)

# preds2 = pipe2.predict(X_test)


create pipeline 2
fit pipeline 2
                      Date Symbol        Low      Close
0      2017-08-09 16:00:00   TATN  391.70000  392.50000
1      2017-08-09 17:00:00   TATN  388.30000  390.65000
2      2017-08-10 10:00:00   TATN  391.35000  393.10000
3      2017-08-10 11:00:00   TATN  390.75000  391.60000
4      2017-08-10 12:00:00   TATN  391.00000  391.90000
...                    ...    ...        ...        ...
74995  2016-11-14 13:00:00   VTBR    0.06858    0.06866
74996  2016-11-14 14:00:00   VTBR    0.06840    0.06935
74997  2016-11-14 15:00:00   VTBR    0.06871    0.06877
74998  2016-11-14 16:00:00   VTBR    0.06850    0.06852
74999  2016-11-14 17:00:00   VTBR    0.06831    0.06836

[75000 rows x 4 columns]


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 143.23it/s]


Pipeline(steps=[('transform_data', ExperimentalTransformer()),
                ('linear_model', LinearRegression())])

In [54]:
with open("D:/logreg_pipeline3.dill", "wb") as f:
    dill.dump(pipe2, f)

In [106]:
import urllib.request
import json      


def get_prediction(x):
    Date, Symbol, Low, Close = x
    body = {'Date': Date, 
            'Symbol': Symbol,
            'Low': Low,
            'Close': Close
            } 

    myurl = "http://127.0.0.1:8180/predict"
    # myurl = "http://127.0.0.1:8180"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    print(jsondata)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

# добавил запрос

In [92]:
import requests
ar = requests.post('http://127.0.0.1:8180/predict', json=X_test[:10].to_json())
ar

<Response [200]>

In [102]:
json.loads(ar.json())[1]

[0.4699121037,
 0.470785284,
 0.4738658043,
 0.4709035918,
 0.4761687491,
 0.4786982541,
 0.4758647877,
 0.4711836916,
 0.4771656822,
 0.4771497252]

In [ ]:
requests.get('http://127.0.0.1:8180/')

In [ ]:
X_test

# дальше не читал) обрати внимание - ниже код отправки запроса изз твоей функции отправляет только название колонок(добавил print в функцию - пытается отправить лишь 
{"Date": "Date", "Symbol": "Symbol", "Low": "Low", "Close": "Close"}

In [107]:
%%time
predictions = get_prediction(X_test[:50])

{"Date": "Date", "Symbol": "Symbol", "Low": "Low", "Close": "Close"}


HTTPError: HTTP Error 500: INTERNAL SERVER ERROR

In [ ]:
predictions

# Павел. Дальше можно не читать

# GENERATE TEST_TRAIN SET FOR MODEL

In [ ]:
def df_data_def(all, features, bars_in_day, label_feature_day, label = 'Short', label_mark = 1):
  # features = ['Min10', 'Min1000']
  # features = ['Cl_min_day']
  target = 'Proft1000' # тут могут быть несколько target - по нему обучаемся, target1 - доп параметр по которому не учимся
  feature_name = []
  name_profit_label = str(target) + '_label'
  name_profit = target
  
  df_data = pd.DataFrame({'Date': all[all[label] == label_mark]['DateTime'].dt.date, 'Symbol': all[all[label] == label_mark]['Symbol'], target: all[all[label] == label_mark][target]}).drop_duplicates()
  df_data['Date'] = pd.to_datetime(df_data['Date'])
  for feature in features:

    new_date = pd.pivot_table(all,
                  index = [all.DateTime.dt.date, all.Symbol],
                  columns = all.DateTime.dt.time,
                  values = feature
                  ).iloc[:,:bars_in_day].reset_index()


  df_data[target] = np.where(df_data[target] > 20, 20, np.where(df_data[target] < -20, -20, df_data[target]))
  df_data[str(target) + '_label'] = np.where(df_data[target] > 0, 1, 0)
  df_data.dropna(inplace=True)
  df_data.sort_values(by = ['Date', 'Symbol'], inplace=True, ignore_index=True)
  display(df_data.shape)
  return df_data, feature_name, name_profit_label, name_profit, target, features

label_mark = 1
bars_in_day = 9
label = 'Short'
features = ['Cl_min_day']
df_data, feature_name, name_profit_label, name_profit, target, features  = df_data_def(all, bars_in_day=bars_in_day, label = label, label_mark = label_mark, label_feature_day = 'today', features = features)

df_data_conect = all[all[label] == label_mark].copy()


In [ ]:
all

In [ ]:
# df_data.to_excel(DIR + 'delete.xlsx')

# Подгонка


In [ ]:
# feature_multy, x_train, y_train, x_test, y_test  = concat_of_split_1year(df_data_blue, year_from = 2019, axis = 2, lag = 3)
feature_multy, x_train, y_train, x_test, y_test  = reshape_concat_of_split_1year(df_data, feature_name=feature_name, year_from = 2019)

print(f'Check: {feature_multy.shape, x_train.shape, x_test.shape}')

In [ ]:
model_param = str('_'.join(features))
shape_set = str(x_train.shape[-2]) + '_' + str(x_train.shape[-1])
proba_1 = model_param + '_' + shape_set
print(str('_'.join(features)), shape_set)

# Model

### NeMod

In [ ]:
feature_multy, x_train, y_train, x_test, y_test  = concat_of_split_1year(df_data, year_from = 2014, axis = 2, lag = 0)

### GB

In [ ]:
from xgboost import XGBClassifier
# fit model no training data
model = XGBClassifier()

#### Train

In [ ]:
# надо ли перемешивать на тесте чтоы при батче 64 мы дивгали веса участь на всех бумажах а не на одной

n_classes = len(np.unique(y_train))
matrix_all = pd.DataFrame()
# epochs = 100
# patience = 10
start_year = 2011
end_year = 2021 + 1 


for year_from in range(start_year, end_year, 1):  
  lag = 'all'
  feature_multy, x_train, y_train, x_test, y_test  = concat_of_split_1year(df_data, year_from = year_from, axis = 2, lag = lag)
  x_train = x_train.reshape(-1, 9*4)
  x_test = x_test.reshape(-1, 9*4)
  input_shape = x_train.shape[1:]
    
  # checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/ROBO/My_Feature_from_NET/keras/' + dir_model + model_param + '_' + shape_set + '/' + str(year_from)
  checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/ROBO/My_Feature_from_NET/XGB/keras_from10_1830to60min/40/Gep/'\
                         + model_param + '_' + shape_set + '/'  + str(year_from)
  print(checkpoint_filepath)
  print(f'Test year: {year_from}')
########################

  model.fit(
      x_train,
      y_train,
      )

  # matrix_year = df_data[(df_data['Date'].dt.year == year_from)].copy()
  # matrix_year[proba_1] = model.predict_proba(x_test)[:,-1]
  # matrix_all = matrix_all.append(matrix_year)
  # print(matrix_all.shape, matrix_year.shape)

  # save
  try:
      os.stat(checkpoint_filepath)
  except:
      os.makedirs(checkpoint_filepath)
  joblib.dump(model, checkpoint_filepath + '/CLF.joblib')

### RandomFores

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib

In [ ]:
model = RandomForestClassifier(n_estimators=20, random_state=42, max_depth=4)

#### Train

In [ ]:
# надо ли перемешивать на тесте чтоы при батче 64 мы дивгали веса участь на всех бумажах а не на одной

n_classes = len(np.unique(y_train))
matrix_all = pd.DataFrame()
# epochs = 100
# patience = 10
start_year = 2011
end_year = 2021 + 1 


for year_from in range(start_year, end_year, 1):  
  lag = 'all'
  feature_multy, x_train, y_train, x_test, y_test  = concat_of_split_1year(df_data, year_from = year_from, axis = 2, lag = lag)
  x_train = x_train.reshape(-1, 9*4)
  x_test = x_test.reshape(-1, 9*4)
  input_shape = x_train.shape[1:]
    
  # checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/ROBO/My_Feature_from_NET/keras/' + dir_model + model_param + '_' + shape_set + '/' + str(year_from)
  checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/ROBO/My_Feature_from_NET/RFC_20est/keras_from10_1830to60min/40/Gep/'\
                         + model_param + '_' + shape_set + '/'  + str(year_from)
  print(checkpoint_filepath)
  print(f'Test year: {year_from}')
########################

  model.fit(
      x_train,
      y_train,
      )

  # matrix_year = df_data[(df_data['Date'].dt.year == year_from)].copy()
  # matrix_year[proba_1] = model.predict_proba(x_test)[:,-1]
  # matrix_all = matrix_all.append(matrix_year)
  # print(matrix_all.shape, matrix_year.shape)

  # save
  try:
      os.stat(checkpoint_filepath)
  except:
      os.makedirs(checkpoint_filepath)
  joblib.dump(model, checkpoint_filepath + '/CLF.joblib')

## Keras LSTM

### Build the model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units = 1))

# out = model(x_train)
# out.shape

### Train

In [ ]:
# надо ли перемешивать на тесте чтоы при батче 64 мы дивгали веса участь на всех бумажах а не на одной

n_classes = len(np.unique(y_train))
matrix_all = pd.DataFrame()
epochs = 100
patience = 10
start_year = 2011
end_year = 2021 + 1 


for year_from in range(start_year, end_year, 1):  
  lag = 'all'
  feature_multy, x_train, y_train, x_test, y_test  = concat_of_split_1year(df_data, year_from = year_from, axis = 2, lag = lag)
  input_shape = x_train.shape[1:]
    
  # checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/ROBO/My_Feature_from_NET/keras/' + dir_model + model_param + '_' + shape_set + '/' + str(year_from)
  checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/ROBO/My_Feature_from_NET/keras_lstm/keras_from10_1830to60min/40/Gep/'\
                         + model_param + '_' + shape_set + '/'  + str(year_from)
  print(checkpoint_filepath)
  print(f'Test year: {year_from}')
########################

  checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, 
                              monitor='val_accuracy',
                              verbose=1, 
                              save_best_only=True,
                              mode='auto')
  EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience = patience, restore_best_weights=True)
  # model = build_model(
  #     input_shape,
  #     head_size=40,
  #     num_heads=8,
  #     ff_dim=4,
  #     num_transformer_blocks=4,
  #     mlp_units=[128],
  #     mlp_dropout=0.4,
  #     dropout=0.25,
  # )

  model.compile(
      loss="binary_crossentropy",
      optimizer=keras.optimizers.Adam(learning_rate=1e-4),
      metrics=['accuracy'],
  )
  # model.summary()

  callbacks = [checkpoint, EarlyStopping]

  model.fit(
      x_train,
      y_train,
      validation_split=0.2,
      epochs=epochs,
      batch_size=64,
      callbacks=callbacks)

  model.evaluate(x_test, y_test, verbose=1)

  matrix_year = df_data[(df_data['Date'].dt.year == year_from)].copy()
  matrix_year[proba_1] = model.predict(x_test)[:,-1]
  matrix_all = matrix_all.append(matrix_year)
  print(matrix_all.shape, matrix_year.shape)

## Pytorch LSTM

In [ ]:
class Dataset_my(torch.utils.data.Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return {'sample': self.x[idx,:, :], 'target': self.y[idx]}

### LSTM

In [ ]:
class My_LSTM(nn.Module):
  def __init__(self, input_size, seq_length, hidden_size, num_layers, num_classes):
    super(My_LSTM, self).__init__()
    self.input_size = input_size
    self.seq_length = seq_length
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.num_classes = num_classes
    self.lstm = nn.LSTM(input_size = self.input_size, hidden_size = self.hidden_size, num_layers = self.num_layers, batch_first = True)
    self.linear = nn.Linear(self.hidden_size, self.num_classes)
    self.soft = nn.Softmax(dim=1)

  def forward(self, x):
    # input.shape -> (batch_size, seq, input_size)
    # h_0.shape -> (num_layer, batch, hidden_size)
    # h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
    # c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
    # Initializing hidden state for first input with zeros
    h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
    # Initializing cell state for first input with zeros
    c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()

    out, _ = self.lstm(x, (h_0, c_0))
    out = out[:, -1, :]    
    out = self.linear(out)
    out = self.soft(out)
    return out
    
    # pass

In [ ]:
# Hyper-parameters 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# input_size = 784 # 28x28
num_classes = 2
num_epochs = 30
batch_size = 128
learning_rate = 0.1

input_size = 4
sequence_length = 9
hidden_size = 50
num_layers = 3

model = My_LSTM(input_size=input_size, seq_length=sequence_length, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
outP = model(torch.tensor(x_train).float())
outP.shape

In [ ]:
outP = outP.detach().numpy()

In [ ]:
outP - out

### CNN

In [ ]:
class CNN_classifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Sequential(nn.Conv1d(9, 4, kernel_size=2),
                                    nn.ReLU(True),
                                    nn.MaxPool1d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(nn.Conv1d(4, 32, kernel_size=3),
                                    nn.ReLU(True),
                                    nn.MaxPool1d(kernel_size=3, stride=2))
        
        # Расчет входных признаков по формуле Lout=((Lin+2*pading - dilation*(kernel - 1) - 1)/stride) + 1
        self.classifier = nn.Sequential(nn.Linear(in_features=906*32, out_features=512, bias=True),
                                        nn.ReLU(True),
                                        nn.Dropout(),
                                        nn.Linear(in_features=512, out_features=512, bias=True),
                                        nn.ReLU(True),
                                        nn.Dropout(),
                                        nn.Linear(512, 2))
    def forward(self, x):
        out = self.layer1(x) 
        # out = self.layer2(out)
        print(out.shape) 
        out = out.reshape(out.size(0), -1) 
        out = self.classifier(out) 
        return out

model_CNN = CNN_classifier()
# model = model.float()

# Параметры обучения
num_epochs = 3
num_classes = 2
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_CNN.parameters(), lr=learning_rate)

In [ ]:
model_CNN(set_['sample'].float())

In [ ]:
set_['sample'].float().shape

### Train

In [ ]:
matrix_all = pd.DataFrame()
start_year = 2011
end_year = 2021 + 1 
lag = 'all'


for year_from in range(start_year, end_year, 1): 
  n = 0 
  acc_all = []
  feature_multy, x_train, y_train, x_test__, y_test__  = concat_of_split_1year(df_data_blue, year_from = year_from, axis = 2, lag = lag)
  # input_shape = x_train.shape[1:]
  x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, shuffle=True, random_state=42)
  # print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

  train_dataset = Dataset_my(x_train, y_train)
  test_dataset  = Dataset_my(x_test,  y_test)

  train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=False)

  test_loader  = torch.utils.data.DataLoader(dataset=test_dataset, 
                                            batch_size=len(test_dataset), 
                                            shuffle=False)

####################### Name dir
  # type_date = 'no_last'
  # if type_date == 'last':
  #   type_dir = 'last_date/'
  # else:
  #   type_dir = 'list_date/'
  
  number_of_back = str(x_test.shape[1])  
  checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/ROBO/My_Feature_from_NET/pytorch/keras_from10_1830to60min/40/Gep/'\
                         + model_param + '_' + shape_set + '/'  + str(year_from)
  print(checkpoint_filepath)
  print(f'Test year: {year_from}')

  # Train the model
  n_total_steps = len(train_loader)
  for epoch in range(num_epochs):
      print(f'epoch: {epoch}')
      for i, set_ in enumerate(train_loader): 
          # break 
          model.train()
          outputs = model(set_['sample'].float())
          loss = criterion(outputs, set_['target'])
          
          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          
          if (i+1) % 100 == 0:
              print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
          # print('Wanna to see diffrences')
          # print(model(torch.tensor(x_test__).float()).shape, model(torch.tensor(feature_multy[:,:,:]).float()))
      # Test the model
      # In test phase, we don't need to compute gradients (for memory efficiency)
      with torch.no_grad():
          n_correct = 0
          n_samples = 0
          for i, set_ in enumerate(test_loader):
              model.eval()
              outputs = model(set_['sample'].float())
              labels = set_['target']
              # max returns (value ,index)
              _, predicted = torch.max(outputs.data, 1)
              n_samples += labels.size(0)
              n_correct += (predicted == labels).sum().item()

          acc = 100.0 * n_correct / n_samples
          acc_all.append(acc)
          print(f'Accuracy of the network: {acc} %')
          if len(acc_all) != 1:
            if acc > max(acc_all[:-1]):     
              print(f'Save model corse: {acc} more {acc_all} loss: {loss}')
              # save
              torch.save(model, checkpoint_filepath)
          else:
            print(f'Save model corse: {acc} more {acc_all} loss: {loss}')  
            # save
            torch.save(model, checkpoint_filepath)
          
  # matrix_year = df_data[(df_data['Date'].dt.year == year_from)].copy()
  # matrix_year[proba_1] = model.predict(x_test)[:,-1]
  # matrix_all = matrix_all.append(matrix_year)
  # print(matrix_all.shape, matrix_year.shape)

## Keras Transformer

### Build the model

Our model processes a tensor of shape `(batch size, sequence length, features)`,
where `sequence length` is the number of time steps and `features` is each input
timeseries.

You can replace your classification RNN layers with this one: the
inputs are fully compatible!

We include residual connections, layer normalization, and dropout.
The resulting layer can be stacked multiple times.

The projection layers are implemented through `keras.layers.Conv1D`.

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs) 

The main part of our model is now complete. We can stack multiple of those
`transformer_encoder` blocks and we can also proceed to add the final
Multi-Layer Perceptron classification head. Apart from a stack of `Dense`
layers, we need to reduce the output tensor of the `TransformerEncoder` part of
our model down to a vector of features for each data point in the current
batch. A common way to achieve this is to use a pooling layer. For
this example, a `GlobalAveragePooling1D` layer is sufficient.

### Train

In [ ]:
# надо ли перемешивать на тесте чтоы при батче 64 мы дивгали веса участь на всех бумажах а не на одной

n_classes = len(np.unique(y_train))
matrix_all = pd.DataFrame()
epochs = 100
patience = 10
start_year = 2015
end_year = 2021 + 1 


for year_from in range(start_year, end_year, 1):  
  lag = 'all'
  # feature_multy, x_train, y_train, x_test, y_test  = concat_of_split_1year(df_data, year_from = year_from, axis = 2, lag = lag)     # переход к reshape_concat_of_split_1year
  feature_multy, x_train, y_train, x_test, y_test  = reshape_concat_of_split_1year(df_data, feature_name=feature_name, year_from = year_from)
  input_shape = x_train.shape[1:]

####################### Name dir
  # type_date = 'no_last'
  # if type_date == 'last':
  #   type_dir = 'last_date/'
  # else:
  #   type_dir = 'list_date/'
  
  # number_of_back = str(x_test.shape[1])  
  # dir_model = 'keras_30min1830/' + type_dir + 'gep1030_year_sicle/' + str(f'lag_{lag}') + '/blue/'    
  # checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/ROBO/My_Feature_from_NET/keras/' + dir_model + model_param + '_' + shape_set + '/' + str(year_from)
  checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/ROBO/My_Feature_from_NET/keras/keras_from10_1830to60min/40/Gep/'\
                         + model_param + '_' + shape_set + '/'  + str(year_from)
  print(checkpoint_filepath)
  print(f'Test year: {year_from}')
########################

  checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, 
                              monitor='val_accuracy',
                              verbose=1, 
                              save_best_only=True,
                              mode='auto')
  EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience = patience, restore_best_weights=True)
  model = build_model(
      input_shape,
      head_size=40,
      num_heads=8,
      ff_dim=4,
      num_transformer_blocks=4,
      mlp_units=[128],
      mlp_dropout=0.4,
      dropout=0.25,
  )

  model.compile(
      loss="sparse_categorical_crossentropy",
      optimizer=keras.optimizers.Adam(learning_rate=1e-4),
      metrics=['sparse_categorical_accuracy', 'accuracy'],
  )
  # model.summary()

  callbacks = [checkpoint, EarlyStopping]

  model.fit(
      x_train,
      y_train,
      validation_split=0.2,
      epochs=epochs,
      batch_size=64,
      callbacks=callbacks)

  model.evaluate(x_test, y_test, verbose=1)

  matrix_year = df_data[(df_data['Date'].dt.year == year_from)].copy()
  matrix_year[proba_1] = model.predict(x_test)[:,-1]
  matrix_all = matrix_all.append(matrix_year)
  print(matrix_all.shape, matrix_year.shape)

# Garbige
# Drop_duplicate <----- df_date_conect

In [ ]:
quant = 20

defi = feature_name + [name_profit] + [name_profit_label] + [proba_1]
new_date = df_data_conect.copy()
new_date[proba_1 + '_quant_' + str(quant)] = pd.qcut(new_date[proba_1], quant, labels=False)
pivo_quant = pd.pivot_table(new_date,
               index = [proba_1 + '_quant_' + str(quant)],
               values = [name_profit] + feature_name,
               aggfunc = ['mean', 'count'],
              #  margins = True
               ).T.drop_duplicates().T
pivo_quant

In [ ]:
#LONG ONLY
treshold = 0.6
treshold_short = 0.45
for_treshold = proba_ful

df_data_conect.Date = pd.to_datetime(df_data_conect.Date)
pivo_long = pd.pivot_table(df_data_conect[(df_data_conect[for_treshold].mean(axis = 1) > treshold)],
               index = [df_data_conect.Date.dt.date],
               values = [name_profit],
               aggfunc = ['mean', 'count'],
               margins = True
               ).T.drop_duplicates().T
display(pivo_long)
pivo_long = pivo_long.iloc[:-1,:]
pivo_long.reset_index(inplace = True)
pivo_long.columns = ['Date', 'mean', 'count']
pivo_long['Date'] = pd.to_datetime(pivo_long['Date'])
pd.pivot_table(pivo_long,
               index = pivo_long.Date.dt.year,
               values = 'mean',
               aggfunc = ['count', 'mean'],
              margins = True
               )

In [ ]:
# Long/Short == long1

def only_long1(df_data_conect, name_profit, enter_long, enter_shrt, for_treshold, from_year, not_take):
  # not_take = 'not_first'
  # enter_long = 0.6
  # enter_shrt = -0.4
  # for_treshold = 'Min10_Min1000_Cl_max_day_Cl_min_day_4_17'

  df_data_conect['Long_Short'] = np.where(df_data_conect[for_treshold] > enter_long, 1, np.where(df_data_conect[for_treshold] < enter_shrt, -1, -10))
  df_data_conect['profit'] = df_data_conect[name_profit]*df_data_conect['Long_Short']
  df_data_conect['proba_1_short_long'] = df_data_conect[for_treshold]*df_data_conect['Long_Short']

  long_short_equity = df_data_conect.sort_values(['Date', 'Long_Short', 'proba_1_short_long'], ascending = [True, False, False], ignore_index=True)
  long_short_equity['Long1'] = np.where(df_data_conect.Date.dt.date.shift(0) != df_data_conect.Date.dt.date.shift(1), 'first', 'not_first')
  long_short_equity = long_short_equity[(long_short_equity['Long_Short'] != -10)&(long_short_equity['Long1'] != not_take)]


  # df_data_conect.Date = pd.to_datetime(df_data_conect.Date)
  pivo = pd.pivot_table(long_short_equity[long_short_equity.Date.dt.year >= from_year],
                index = [long_short_equity.Date.dt.date],
                values = ['profit'],
                aggfunc = ['mean', 'count'],
                margins = True
                ).T.drop_duplicates().T
  display(pivo)
  pivo.reset_index(inplace = True)
  pivo = pivo.iloc[:-1,:]
  pivo.columns = ['Date', 'mean', 'count']
  pivo['Date'] = pd.to_datetime(pivo['Date'])
  display(pd.pivot_table(pivo,
                index = pivo.Date.dt.year,
                values = 'mean',
                aggfunc = ['count', 'mean'],
                margins = True
                ))
##########################################  
not_take = 'not_first'
enter_long = 0.6
enter_shrt = -0.4
for_treshold = proba_ful[0]
from_year = 2011
for_profit = 'profit_1hour'
print(f'for_profit: {for_profit}')

# df_data_conect['Long_Short'] = np.where(df_data_conect[proba_1] > enter_long, 1, np.where(df_data_conect[proba_1] < enter_shrt, -1, -10))
# df_data_conect['profit'] = df_data_conect[name_profit]*df_data_conect['Long_Short']
# df_data_conect['proba_1_short_long'] = df_data_conect[proba_1]*df_data_conect['Long_Short']

# long_short_equity = df_data_conect.sort_values(['Date', 'Long_Short', 'proba_1_short_long'], ascending = [True, False, False], ignore_index=True)
# long_short_equity['Long1'] = np.where(df_data_conect.Date.dt.date.shift(0) != df_data_conect.Date.dt.date.shift(1), 'first', 'not_first')

only_long1(df_data_conect, for_profit, enter_long = enter_long, enter_shrt = enter_shrt, for_treshold = for_treshold, from_year = from_year, not_take = not_take)

In [ ]:
#  Предсказание на 30 минутках в ЦИКЛЕ
# count	mean
# mean	mean
# Date		
# 2017	75	0.500150
# 2018	92	0.405941
# 2019	42	0.582666
# 2020	171	0.633090
# 2021	140	0.508307
# All	520	0.536060

In [ ]:
# SHORT ONLY
df_data_conect.Date = pd.to_datetime(df_data_conect.Date)
pivo_short = pd.pivot_table(df_data_conect[(df_data_conect[proba_1] < 0.4)],
               index = [df_data_conect.Date.dt.date],
               values = [name_profit],
               aggfunc = ['mean', 'count'],
               margins = True
               ).T.drop_duplicates().T
display(pivo_short)
pivo_short = pivo_short.iloc[:-1,:]
pivo_short.reset_index(inplace = True)
pivo_short.columns = ['Date', 'mean', 'count']
pivo_short['Date'] = pd.to_datetime(pivo_short['Date'])
pd.pivot_table(pivo_short,
               index = pivo_short.Date.dt.year,
               values = 'mean',
               aggfunc = ['count', 'mean'],
              margins = True
               )

### Long and Short

In [ ]:
enter_long = 0.6
enter_shrt = 0.45
not_take = 'not_first'

df_data_conect['Long_Short'] = np.where(df_data_conect[proba_1] > enter_long, 1, np.where(df_data_conect[proba_1] < enter_shrt, -1, -10))
df_data_conect['profit'] = df_data_conect['profit_GEP']*df_data_conect['Long_Short']
df_data_conect['proba_1_short_long'] = df_data_conect[proba_1]*df_data_conect['Long_Short']

long_short_equity = df_data_conect.sort_values(['Date', 'Long_Short', 'proba_1_short_long'], ascending = [True, False, False], ignore_index=True)
long_short_equity['Long1'] = np.where(df_data_conect.Date.dt.date.shift(0) > df_data_conect.Date.dt.date.shift(1), 'first', 'not_first')
long_short_equity = long_short_equity[(long_short_equity['Long_Short'] != -10)&(long_short_equity['Long1'] != not_take)]


# df_data_conect.Date = pd.to_datetime(df_data_conect.Date)
pivo = pd.pivot_table(long_short_equity,
               index = [long_short_equity.Date.dt.date],
               values = ['profit'],
               aggfunc = ['mean', 'count'],
               margins = True
               ).T.drop_duplicates().T
display(pivo)
pivo.reset_index(inplace = True)
pivo = pivo.iloc[:-1,:]
pivo.columns = ['Date', 'mean', 'count']
pivo['Date'] = pd.to_datetime(pivo['Date'])
pd.pivot_table(pivo,
               index = pivo.Date.dt.year,
               values = 'mean',
               aggfunc = ['count', 'mean'],
              margins = True
               )

# Plot violone

In [ ]:
# large = 22; med = 16; small = 12
# params = {'axes.titlesize': large,
#           'legend.fontsize': med,
#           'figure.figsize': (4, 10),
#           'axes.labelsize': med,
#           'axes.titlesize': med,
#           'xtick.labelsize': med,
#           'ytick.labelsize': med,
#           'figure.titlesize': large}
# plt.rcParams.update(params)
# plt.style.use('seaborn-whitegrid')
# sns.set_style("white")
# %matplotlib inline

plt.figure(figsize=(20,10), dpi= 80)
ax = sns.violinplot(x=new_date[proba_1 + '_quant_' + str(quant)], y = new_date[name_feature[1]],  scale='width', inner='quartile')

# Оценка результата

## Склеиваем сделки

In [ ]:
df_f_all = matrix_year[matrix_year['Symbol'] != 'fGAZP'].copy()
### СОРТИРОВАТЬ ПО SYMBOL!!!
threshold_in = 0.53
threshold_out = 0.50


def mark(df_f_all, threshold_in = 0.52, threshold_out = 0.50, stop = False):
  hour_np = np.array(df_f_all['Date'].dt.hour)
  close_np = np.array(df_f_all['Close'])
  proba2_range_np = np.array(df_f_all['proba2_range'])
  min10_np = np.array(df_f_all['Min10'])
  pr1000_np = np.array(df_f_all['Proft1000_norm'])
  signal = np.zeros(shape=len(df_f_all))
  close_hour = np.zeros(shape=len(df_f_all))
  close_hour_1 = np.zeros(shape=len(df_f_all))
  min_enter = np.zeros(shape=len(df_f_all))
  pr1000_enter = np.zeros(shape=len(df_f_all))
  profit_hour_np = np.zeros(shape=len(df_f_all))
  profit_hour_np_1 = np.zeros(shape=len(df_f_all))
  hold = np.zeros(shape=len(df_f_all))
  hold_1 = np.zeros(shape=len(df_f_all))


  for i in range(close_np.size):
    # print(proba2_range_np[i], i)
    # print(10*('ddddd'))
    if i == 0:
      # print(proba2_range_np[i])
      signal[i] = 0
    elif proba2_range_np[i] > threshold_in and (signal[i-1] == 0 or signal[i-1] == 3):
      # print(proba2_range_np[i])
      signal[i] = 1
      hold[i] = 1
    elif stop and hour_np[i] == 10 and (signal[i-1] == 2 or signal[i-1] == 1):      
      signal[i] = 3
      hold[i] = hold[i-1]
    elif proba2_range_np[i] > threshold_out and (signal[i-1] == 2 or signal[i-1] == 1):
      signal[i] = 2
      hold[i] = 1 + hold[i-1]      
    elif (proba2_range_np[i] <= threshold_out and (signal[i-1] == 2 or signal[i-1] == 1)):
      signal[i] = 3
      hold[i] = hold[i-1]
    elif proba2_range_np[i] <= threshold_in and (signal[i-1] == 0 or signal[i-1] == 3):
      # print(proba2_range_np[i])
      signal[i] = 0
      hold[i] = 0

  df_f_all['signal'] = signal
  df_f_all['hold'] = hold

  for i in range(close_np.size):
    # print(proba2_range_np[i], i)
    # print(10*('ddddd'))
    if i == 0 or signal[i] == 0:
      close_hour[i] = None
      min_enter[i] = None
      pr1000_enter[i] = None
      profit_hour_np[i] = (close_np[i]/close_hour[i] - 1)*100
         
    elif signal[i] == 1:
      close_hour[i] = close_np[i]
      min_enter[i]  = min10_np[i]
      pr1000_enter[i]   = pr1000_np[i]
      profit_hour_np[i] = None
    else:
      close_hour[i] = close_hour[i-1]
      min_enter[i]  = min_enter[i-1]
      pr1000_enter[i]   = pr1000_enter[i-1]
      profit_hour_np[i] = (close_np[i]/close_hour[i] - 1)*100

    # for i in range(close_np.size-1, -1, -1):
    #   if signal[i] == 3 or i == close_np.size-1:
    #     close_hour_1[i] = close_np[i]
    #     profit_hour_np_1[i] = None
    #   else:
    #     close_hour_1[i] = close_hour_1[i+1]
    #     profit_hour_np_1[i] = (close_hour_1[i]/close_np[i] - 1)*100

  close_np_1 = close_np[::-1]
  signal_1 = signal[::-1]
  for i in range(close_np.size):
    if signal_1[i] == 3 or i == 0:
      close_hour_1[i] = close_np_1[i]
      profit_hour_np_1[i] = None
      hold_1[i] = 0
    else:
      close_hour_1[i] = close_hour_1[i-1]
      profit_hour_np_1[i] = (close_hour_1[i]/close_np_1[i] - 1)*100
      hold_1[i] = hold_1[i-1] + 1


  df_f_all['close_hour'] = close_hour
  df_f_all['profit_hour_np'] = profit_hour_np
  df_f_all['profit_hour_np'] = np.where(df_f_all['profit_hour_np'] > 20, 20, np.where(df_f_all['profit_hour_np'] < -20, -20, df_f_all['profit_hour_np']))
  df_f_all['min_enter'] = min_enter
  df_f_all['pr1000_enter'] = pr1000_enter

  
  df_f_all['profit_hour_np_1'] = profit_hour_np_1[::-1]
  df_f_all['profit_hour_np_1'] = np.where(df_f_all['profit_hour_np_1'] > 20, 20, np.where(df_f_all['profit_hour_np_1'] < -20, -20, df_f_all['profit_hour_np_1']))
  df_f_all['close_hour_1'] = close_hour_1[::-1]
  df_f_all['hold_1'] = hold_1[::-1]

  return df_f_all

df_f_all = mark(df_f_all, threshold_in = threshold_in, threshold_out = threshold_out, stop = True)

In [ ]:
pd.pivot_table(df_f_all[(df_f_all['signal'] == 1)&(df_f_all['Year'] > -2014)],
               index = df_f_all.Date.dt.hour,
               values = ['hold_1', 'Min1000', 'min_enter', 'Proft1000_norm', 'pr1000_enter', 'profit_hour_np_1', 'Proft1000_label'],
               aggfunc=['count', 'mean'],
               margins=True
               ).T.drop_duplicates().T

In [ ]:
# signal = 3
pd.pivot_table(df_f_all[(df_f_all['signal'] == 3)&(df_f_all['Year'] > -2014)],
               index = df_f_all.Date.dt.hour,
               values = ['signal', 'profit_hour_np', 'hold', 'Min1000', 'min_enter', 'Proft1000_norm', 'pr1000_enter', 'proba2_range'],
               aggfunc=['count', 'mean'],
               margins=True
               ).T.drop_duplicates().T

## Plot

In [ ]:
from_year = 2010
comission = 0.1/100
name_for_p = ['pivo_long', 'pivo_short', 'pivo']
equity_dic = dict.fromkeys(name_for_p)


for p, name_p in zip ([pivo_long, pivo_short, pivo], name_for_p):
  print(f'Сделки: {name_p}')
  pivo_for_plot = p[p.Date.dt.year >= from_year]  
  equity = np.zeros(shape = len(pivo_for_plot.iloc[:-1,:]))
  equity_reinvest = np.zeros(shape = len(pivo_for_plot.iloc[:-1,:]))
  if name_p != 'pivo_short':
    profit_hour_np_1 = np.array(pivo_for_plot[('mean')])[:-1]
  else:
    profit_hour_np_1 = np.array(-pivo_for_plot[('mean')])[:-1]

  for i in range(len(equity)):
    if i == 0:
      equity[i] = 100 + 100*(profit_hour_np_1[i]/100 - comission)
      equity_reinvest[i] = 100*(1 + profit_hour_np_1[i]/100 - comission)
    else:
      equity[i] = equity[i-1] + 100*(profit_hour_np_1[i]/100 - comission)
      equity_reinvest[i] = equity_reinvest[i-1]*(1 + profit_hour_np_1[i]/100 - comission)

  # скачали индексы
  list_index = dict.fromkeys(['SANDP', 'IMOEX', 'MOEX10', 'RTS'])
  # for i, keys in zip(os.listdir(DIR + 'Data/all/index/'), ['SANDP', 'IMOEX', 'MOEX10', 'RTS']):
  for i, keys in zip(['IMOEX.csv'], ['IMOEX']):
    list_index[keys] = pd.read_csv(DIR + 'Data/all/index/' + i, parse_dates=['<DATE>'])
    list_index[keys]['Year'] = list_index[keys]['<DATE>'].dt.year

  # построили df_equity
  df_equity = pd.DataFrame([np.array(pivo_for_plot['Date']), equity[:]]).T
  df_equity.columns = ['<DATE>', 'equity']
  df_equity['<DATE>'] = pd.to_datetime(df_equity['<DATE>']).dt.date
  df_equity['<DATE>'] = pd.to_datetime(df_equity['<DATE>'])

  # обьеденили 
  for_plot = pd.merge(list_index['IMOEX'][list_index['IMOEX']['Year']>=from_year], df_equity, on = '<DATE>', how = 'left')
  # for_plot = pd.merge(for_plot, list_index['MOEX10'], on = '<DATE>', how = 'left', suffixes=('', 'MOEX10'))
  for_plot['equity'].fillna(method='ffill', inplace=True)
  equity_dic[name_p] = list(for_plot['equity'])
  

  # Plot
  plt.figure(figsize=(20, 10))
  plt.plot(for_plot['<DATE>'], for_plot['equity'], label='my_equity')
  # plt.plot(for_plot['<DATE>'], equity_max, label='my_equity')
  plt.plot(for_plot['<DATE>'], for_plot['<CLOSE>']*100/for_plot['<CLOSE>'][0], label='IMOEX')
  plt.legend()
  plt.show()

## Метрики WL

In [ ]:
# name_for_p = ['pivo_long', 'pivo_short', 'pivo']
equity_analize = 'equity'
for eq in name_for_p:
  print(eq)
  # equity_from_plot = for_plot[equity_analize]
  equity_from_plot = pd.Series(equity_dic[eq])
  equity_max = np.zeros(shape = len(equity_from_plot))

  for i in range(len(equity_max)):
    if i == 0:
      equity_max[i] = equity_from_plot[i]
    else:
      equity_max[i] = (equity_from_plot[:i+1]).max()

  for_plot['equity_max'] = equity_max

  ddown_bar = []
  for i in range(len(for_plot['equity'])):
    if i == 0:
      k = 0
    elif equity_max[i] == equity_max[i - 1]:
      k += 1
    else:
      k = 1
    ddown_bar.append(k)
    # print(k)

  for_plot['DrowDown_bar'] = ddown_bar

  for_plot['DrowDown'] = (for_plot['equity_max'] / equity_from_plot - 1) * 100

  # Количество дней до нового максимума
  plt.figure(figsize=(20, 10))
  plt.plot(for_plot['<DATE>'], for_plot['DrowDown_bar'])
  plt.show()

  # Просадка счета от последнего хая
  plt.figure(figsize=(20, 10))
  plt.plot(for_plot['<DATE>'], for_plot['DrowDown'])
  plt.show()

  # % Profit Distribution of Trades

  plt.figure(figsize=(20, 10))
  plt.hist(pivo[('mean')], bins = 100)
  plt.show()
  # break

In [ ]:
# Win_rate
win_rate = np.where(pivo[('mean')] >= 0, 1, 0)
win_rate = round(win_rate.mean()*100, 2)
loss_rate = round(100 - win_rate, 2)
print('Win_rate')
display(win_rate, loss_rate)

# Прибыль на 1 сделку:
Profit_trade = round(pivo[('mean')].mean(), 2)
print('Прибыль на 1 сделку')
display(Profit_trade)

# Средняя прибыль в процентах на прибыльную и убыток на убыточную сделку
Average_profit = round(pivo[pivo[('mean')]  > 0][[('mean')]].mean(), 2)
Average_loss   = round(pivo[pivo[('mean')] <= 0][[('mean')]].mean(), 2)
print('Profit_trade, Average_profit, Average_loss')
display(Profit_trade, Average_profit, Average_loss)

# Профит фактор
profit_factor = (pivo[pivo[('mean')] > 0][[('mean')]].sum()/-pivo[pivo[('mean')] <= 0][[('mean')]].sum()).round(2)
profit_factor
print('profit_factor')
display(profit_factor)


# Насколько прибыль превышает глубину  максимальной просадки
Recovery_factor = round((for_plot['equity'][-1:] / 100 - 1) * 100 / (for_plot['DrowDown']).max(), 2)
Recovery_factor
print('Recovery_factor')
display(Recovery_factor)

# Cоотношение средней прибыльной сделки к средней убыточной
Payoff_ratio = round(Average_profit/-Average_loss, 2)
Payoff_ratio
print('Payoff_ratio')
display(Payoff_ratio)

# Cluster

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from sklearn.cluster import KMeans

from sklearn.manifold import TSNE

plt.rcParams['figure.figsize'] = [11, 8]

## KMEANS

In [ ]:
all_1 = df_data_conect[df_data_conect[model_param] >= 0.6].copy()
X = all_1[name_feature]
X_all = df_data_conect[name_feature]

In [ ]:
k_means = KMeans(n_clusters = 5)
k_means = k_means.fit(X)
clusters = k_means.predict(X)

In [ ]:
all_1['clusters'] = clusters

display(pd.pivot_table(all_1[all_1['clusters'] != -2],
               index = ['test_train' ,'clusters'],
               values = [name_profit, name_profit_label] + ['Min10_' + str(i) for i in range(10,19)],
               aggfunc = ['count', 'mean'],
              #  margins = True
               ).T.drop_duplicates().T.head(100))

## TSNE

In [ ]:
tsne = TSNE(n_components=2)
new_codes = tsne.fit_transform(X)

all_1['tsne_x'] = new_codes[:,0]
all_1['tsne_y'] = new_codes[:,1]

In [ ]:
for cluster, color in zip(all_1['clusters'].unique(), ['r', 'g', 'blue', 'yellow', 'black']):
  plt.scatter(all_1[all_1['clusters'] == cluster]['tsne_x'], all_1[all_1['clusters'] == cluster]['tsne_y'], c = color)
plt.show()

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.inspection import permutation_importance

rfc = RandomForestClassifier(n_estimators=10, max_depth=20)
rfc.fit(x_train.squeeze(), y_train, )

rt = DecisionTreeClassifier(max_depth=4)
rt.fit(x_train.squeeze(), y_train, )


ful_date = new_date[[str(i) for i in list(range(10,19))]].squeeze()
xx_predRFC = rfc.predict_proba(ful_date)
xx_predRT = rt.predict_proba(ful_date)

metkaRFC = xx_predRFC
metkaRT = xx_predRT
# label = all['Min10_label']
label = new_date['profit_GEP_label']
# label = y_test

treshold = 0.5
tresholdUp = 10.57
# metkaRFC = np.where(metkaRFC[:,1] > 0.5, 1, 0)
# metkaRT = np.where(metkaRT[:,1] > 0.5, 1, 0)

metkaRFC = np.where((metkaRFC[:,1] > treshold) & (metkaRFC[:,1] < tresholdUp), 1, 0)
metkaRT = np.where(metkaRT[:,1] > treshold, 1, 0)


display(confusion_matrix(label, metkaRFC), confusion_matrix(label, metkaRT))


new_date['foreRFC'] = metkaRFC
display(pd.pivot_table(new_date,
               index = ['test_train' ,'foreRFC'],
               values = ['profit_GEP', 'profit_GEP_label'] + [str(i) for i in list(range(10,19))],
               aggfunc = ['count', 'mean']
              #  margins = True
               ).T.drop_duplicates().T.head(100))
